In [1]:
import os
import requests
import bs4
from pathlib import Path

In [3]:
url_bis = r'https://www.bis.org/statistics/full_data_sets.htm'

In [4]:
cont_bis = requests.get(url_bis)

In [6]:
soup = bs4.BeautifulSoup(cont_bis.content)
htm_li_filelist = soup.find(id='cmsContent').find('ul').find('li')

In [9]:
num_files = 0
lst_files = list()
for i, x in enumerate(soup.find(id='cmsContent').find('ul').findAll('li')):
    if i < 3: continue # skip first 3 files(LBS, CBS, DEBT_SEC) because it is big
    url_zipfile = r'https://www.bis.org' + x.find('a')['href']
    zip_content = requests.get(url_zipfile)
    with open(os.path.join('downloads',url_zipfile.split(r'/')[-1]), 'wb') as fout:
        fout.write(zip_content.content)
        print('Downloaded: {0}'.format(url_zipfile))
        num_files = num_files + 1
        lst_files.append(url_zipfile.split(r'/')[-1])
    if num_files == 5: # download only 5 files
        break

Downloaded: https://www.bis.org/statistics/full_bis_total_credit_csv.zip
Downloaded: https://www.bis.org/statistics/full_webstats_credit_gap_dataflow_csv.zip
Downloaded: https://www.bis.org/statistics/full_bis_dsr_csv.zip
Downloaded: https://www.bis.org/statistics/full_bis_gli_csv.zip
Downloaded: https://www.bis.org/statistics/full_bis_xtd_csv.zip


In [12]:
import os
import shutil
from zipfile import ZipFile

In [11]:
lst_files

['full_bis_total_credit_csv.zip',
 'full_webstats_credit_gap_dataflow_csv.zip',
 'full_bis_dsr_csv.zip',
 'full_bis_gli_csv.zip',
 'full_bis_xtd_csv.zip']

In [15]:
lst_unzip_folder = list()
for csvfile in lst_files:
    csvfile = os.path.join(r'downloads', csvfile)
    csvfile_unzip = csvfile + 'unzipped'
    if os.path.exists(csvfile):
        #If the folder is exist, it removes forcefully
        if os.path.isdir(csvfile_unzip):
            shutil.rmtree(csvfile_unzip)
        else:
            os.mkdir(csvfile_unzip)
        with ZipFile(csvfile, 'r') as zipObj:
            # Extract all the contents of zip file in current directory
            zipObj.extractall(csvfile_unzip)
            lst_unzip_folder.append(csvfile_unzip)
            print('Extracted: {0}'.format(csvfile_unzip))

Extracted: downloads\full_bis_total_credit_csv.zipunzipped
Extracted: downloads\full_webstats_credit_gap_dataflow_csv.zipunzipped
Extracted: downloads\full_bis_dsr_csv.zipunzipped
Extracted: downloads\full_bis_gli_csv.zipunzipped
Extracted: downloads\full_bis_xtd_csv.zipunzipped


In [16]:
import pandas as pd

In [18]:
for unzip_folder in lst_unzip_folder:
    for csvfile in os.listdir(unzip_folder):
        csvfile2 = os.path.join(unzip_folder, csvfile)
        df = pd.read_csv(csvfile2)
        print('# row and columns of {0}: {1}'.format(csvfile, df.shape))

# row and columns of WEBSTATS_TOTAL_CREDIT_DATAFLOW_csv_col.csv: (1130, 332)
# row and columns of WEBSTATS_CREDIT_GAP_DATAFLOW_csv_col.csv: (132, 284)
# row and columns of WEBSTATS_DSR_DATAFLOW_csv_col.csv: (66, 89)
# row and columns of WEBSTATS_GLI_DATAFLOW_csv_col.csv: (225, 185)
# row and columns of WEBSTATS_XTD_DATAFLOW_csv_col.csv: (1296, 485)
